# Whatsapp Chat Analysis

In [47]:
import numpy as np
import pandas as pd
import regex
import emoji
from collections import Counter
import matplotlib.pyplot as plt
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Extraction Functions

In [48]:
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = regex.match(pattern, s)
    if result:
        return True
    return False

def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

In [49]:
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

## Importing Data

In [50]:
data = []
conversation = 'WhatsApp Chat with Anubharathy.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [51]:
chat_df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
chat_df['Date'] = pd.to_datetime(chat_df['Date'])

## Exploring Data

In [52]:
chat_df.head()

,Date,Time,Author,Message
0,2021-01-01,10:16 AM,Anubharathy,Tq... Same to u😁
1,2021-01-01,12:15 PM,Veeramanohar,Haa Gm
2,2021-01-01,3:03 PM,Anubharathy,Gm
3,2021-01-01,9:26 PM,Veeramanohar,Oii.. inaiku day epdi pochu ?!
4,2021-01-01,11:05 PM,Anubharathy,Asusual


In [53]:
chat_df.tail()

,Date,Time,Author,Message
11052,2021-10-13,4:07 PM,Veeramanohar,11am ku Gn ah 😂
11053,2021-10-13,4:07 PM,Veeramanohar,Saptiya?
11054,2021-10-13,6:19 PM,Anubharathy,Haa
11055,2021-10-13,6:19 PM,Anubharathy,*gm
11056,2021-10-13,7:40 PM,Veeramanohar,haa


In [54]:
chat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11057 entries, 0 to 11056
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     11057 non-null  datetime64[ns]
 1   Time     11057 non-null  object        
 2   Author   11014 non-null  object        
 3   Message  11057 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 345.7+ KB


In [60]:
authors=chat_df.Author.unique()
authors

array(['Anubharathy', 'Veeramanohar', None], dtype=object)

In [56]:
total_messages=chat_df.shape[0]
total_messages

11057

In [57]:
media_messages = chat_df[chat_df["Message"]=='<Media omitted>'].shape[0]
media_messages

358

### Extracting Emojis

In [64]:
def split_count(text):
    emoji_list = []
    data = regex.findall(r'\X',text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    return emoji_list

chat_df['emoji'] = chat_df["Message"].apply(split_count)

emojis = sum(chat_df['emoji'].str.len())
emojis

0

In [63]:
chat_df.head()

,Date,Time,Author,Message,emoji,urlcount
0,2021-01-01,10:16 AM,Anubharathy,Tq... Same to u😁,[],0
1,2021-01-01,12:15 PM,Veeramanohar,Haa Gm,[],0
2,2021-01-01,3:03 PM,Anubharathy,Gm,[],0
3,2021-01-01,9:26 PM,Veeramanohar,Oii.. inaiku day epdi pochu ?!,[],0
4,2021-01-01,11:05 PM,Anubharathy,Asusual,[],0


### Extracting URLs

In [59]:
URLPATTERN = r'(https?://\S+)'
chat_df['urlcount'] = chat_df.Message.apply(lambda x: regex.findall(URLPATTERN, x)).str.len()
links = np.sum(chat_df.urlcount)
links

35

In [61]:
print(f'Chats between {authors[0]} and {authors[1]}')
print("Total Messages: ", total_messages)
print("Number of Media Shared: ", media_messages)
print("Number of Emojis Shared", emojis)
print("Number of Links Shared", links)

Chats between Anubharathy and Veeramanohar
Total Messages:  11057
Number of Media Shared:  358
Number of Emojis Shared 0
Number of Links Shared 35
